In [2]:
import cloudscraper
import time
from bs4 import BeautifulSoup
import pandas as pd

pd.set_option('max_colwidth', 400)

In [3]:
df = pd.read_csv("property_data.csv", index_col=False)

### 1.0 Remove duplicated data during first scraping

In [4]:
df.head()

,property_name,property_listing,property_type
0,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-may-ng-35025945,condominium
1,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-33616932,condominium
2,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-34630572,condominium
3,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-35076662,condominium
4,For Sale - Middleton @ Minden Heights,https://www.propertyguru.com.my/property-listing/middleton-minden-heights-for-sale-by-jonathan-lee-35397439,condominium


In [5]:
df.shape

(15368, 3)

In [6]:
df = df.drop_duplicates()

In [ ]:
df.shape

(7920, 3)

In [135]:
df.head()

,property_name,property_listing,property_type
0,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-may-ng-35025945,condominium
1,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-33616932,condominium
2,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-34630572,condominium
3,For Sale - Tropicana Bay Residences,https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-35076662,condominium
4,For Sale - Middleton @ Minden Heights,https://www.propertyguru.com.my/property-listing/middleton-minden-heights-for-sale-by-jonathan-lee-35397439,condominium


In [117]:
df = df[~df["property_name"].str.contains("Versa")].reset_index(drop=True)

In [121]:
df = df[~df["property_name"].str.contains("Rubica")].reset_index(drop=True)

In [122]:
df[df["property_name"].str.contains("Rubica")]

,property_name,property_listing,property_type


### 2.0 Scraping with more info

In [9]:
property_type = []
floor_size = []
developer = []
land_Size = []
psf = []
furnishing = []
floor_level = []
tenure = []
built_year = []
listing_id = []
listing_time = []

bedroom = []
bathroom = []
location = []
url_list = []

In [8]:
def BSPrep(URL):
    exitcode = 1
    while exitcode == 1:
        try:
            trial = 0
            while trial < 10:
                scraper = cloudscraper.create_scraper()
                print('Loading '+URL)
                s = scraper.get(URL)
                soup = BeautifulSoup(s.content, 'html.parser')
                if "captcha" in soup.text:
                    trial += 1
                    print('Retrying '+' ('+str(trial)+'/10) ...')
                    time.sleep(0.1)
                    continue
                elif "No Results" in soup.text:
                    print('Invalid URL, skipping '+URL)
                    trial = 99
                else:
                    trial = 99
            if trial == 10:
                print('Trial exceeded, skipping '+URL)
            exitcode = 0
            return soup
        except:
                print('Connection reset, retrying in 1 mins...', flush=True)
                time.sleep(60)
                
def get_property_details(soup,url):
    info_list = []
    if len(soup.find_all( "td", {"class":"value-block"})) < 11:
        pass
    else:
        for row in soup.find_all( "td", {"class":"value-block"}):
            info_list.append(row.text)

        for i in soup.find_all( "span", {"class":"element-label"})[2:4]:
            text = i.text.replace(" ", "")
            text = "".join([s for s in text.splitlines(True) if s.strip("\r\n")])
            info_list.append(text)
    
        property_type.append(info_list[0])
        floor_size.append(info_list[1])
        developer.append(info_list[2])
        land_Size.append(info_list[3])
        psf.append(info_list[4])
        furnishing.append(info_list[5])
        floor_level.append(info_list[6])
        tenure.append(info_list[7])
        built_year.append(info_list[8])
        listing_id.append(info_list[9])
        listing_time.append(info_list[10])
        bedroom.append(info_list[11])
        bathroom.append(info_list[12])
        location.append(soup.find( "span", itemprop="streetAddress").text)
        url_list.append(url)

In [13]:
bs = BSPrep("https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-33616932")

Loading https://www.propertyguru.com.my/property-listing/tropicana-bay-residences-for-sale-by-simon-saw-33616932


In [14]:
len(bs.find_all( "td", {"class":"value-block"}))

11

In [18]:
len(floor_size)

7915

In [33]:
len(url_list)

7915

In [34]:
data= {
    "property_type" : property_type, 
    "floor_size": floor_size,
    "developer": developer,
    "land_size": land_Size,
    "psf" : psf,
    "furnishing" : furnishing,
    "floor_level" : floor_level,
    "tenure_type" : tenure,
    "built_year" : built_year,
    "listing_id" : listing_id,
    "listing_time" : listing_time,
    "bedroom_count" : bedroom,
    "bathroom_counr" : bathroom,
    "location": location,
    "propertyguru_url": url_list
    
}

new_df = pd.DataFrame(data=data)

In [36]:
new_df.shape

(7915, 15)

In [37]:
new_df.to_csv("property_more_info.csv", index=False)